<a href="https://colab.research.google.com/github/MK316/workspace/blob/main/ASR01/ASR04_RecRate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recognition Rate calculation using nltk

Need a recognition result file and rainbow sentence file

* **SE.zip** (gTTS generated audio files) - completed (asof 0106)
* SK.zip (gTTS generated audio files)
* HE.zip (Native production)
* HK.zip (Learners' production)

## Perception rate ~ Error rate

* Perception rate: Percentage of words that match the original text

### Demo

- Sample text: "**The rainbow** is a division of white light into many beautiful colors." (12 words)

- Perceived text: **The lane bowl** is a division of white light into many beautiful colors. (11 words)

Perception rate = (11/12)*100 = 91%  
Error rate = 9%

### Read the rainbow text (19 sentences)

[File from github](https://raw.githubusercontent.com/MK316/workspace/main/ASR01/data/rainbow_sentences.csv)

In [2]:
import pandas as pd

url = "https://raw.githubusercontent.com/MK316/workspace/main/ASR01/data/rainbow_sentences.csv"
rb = pd.read_csv(url)
print(rb.head())

url2 = "https://raw.githubusercontent.com/MK316/workspace/main/ASR01/results/SE_result_0106.csv"
se = pd.read_csv(url2)
print(se.head())



   SN                                           Sentence
0  S1  When the sunlight strikes raindrops in the air...
1  S2  The rainbow is a division of white light into ...
2  S3  These take the shape of a long round arch, wit...
3  S4  There is, according to legend, a boiling pot o...
4  S5             People look, but no one ever finds it.


,Filename,Runtime,Recognized
0,SE01.wav,0.641,When the sunlight strikes raindrops in the ai...
1,SE02.wav,0.324,The rainbow is a division of white light into...
2,SE03.wav,0.397,These take the shape of a long round arch wit...
3,SE04.wav,0.309,"There is, according to legend, a boiling pot ..."
4,SE05.wav,0.261,"People look, but no one ever finds it."


In [12]:
# Add original text to the result file

oritext = list(rb['Sentence'])
sn = list(rb['SN'])

se['Sentence'] = oritext
se['SN'] = sn
se.head()

,SN,Filename,Runtime,Oritext,Recognized,Sentence
0,S1,SE01.wav,0.641,When the sunlight strikes raindrops in the air...,When the sunlight strikes raindrops in the ai...,When the sunlight strikes raindrops in the air...
1,S2,SE02.wav,0.324,The rainbow is a division of white light into ...,The rainbow is a division of white light into...,The rainbow is a division of white light into ...
2,S3,SE03.wav,0.397,"These take the shape of a long round arch, wit...",These take the shape of a long round arch wit...,"These take the shape of a long round arch, wit..."
3,S4,SE04.wav,0.309,"There is, according to legend, a boiling pot o...","There is, according to legend, a boiling pot ...","There is, according to legend, a boiling pot o..."
4,S5,SE05.wav,0.261,"People look, but no one ever finds it.","People look, but no one ever finds it.","People look, but no one ever finds it."


Column ordering & save it as csv (e.g., 'SER_result.csv')

In [13]:
se = se[['SN', 'Filename', 'Runtime', 'Sentence', 'Recognized']]
cols = list(se.columns.values)
se.to_csv('SER_result.csv', index=False)
se.head()

,SN,Filename,Runtime,Sentence,Recognized
0,S1,SE01.wav,0.641,When the sunlight strikes raindrops in the air...,When the sunlight strikes raindrops in the ai...
1,S2,SE02.wav,0.324,The rainbow is a division of white light into ...,The rainbow is a division of white light into...
2,S3,SE03.wav,0.397,"These take the shape of a long round arch, wit...",These take the shape of a long round arch wit...
3,S4,SE04.wav,0.309,"There is, according to legend, a boiling pot o...","There is, according to legend, a boiling pot ..."
4,S5,SE05.wav,0.261,"People look, but no one ever finds it.","People look, but no one ever finds it."


## Recognition rate and finding mismatching words using NLTK

In [67]:
#@markdown Calculate recognition rate, record missing words => dataframe (SETR_result.csv) 
from nltk.tokenize import RegexpTokenizer

nw = []
rw = []
rr = []
mw = []


for i in range(0,len(se['SN'])):
  t1 = se['Sentence'][i]

# text 1
  txt1 = t1.lower()
  tokenizer = RegexpTokenizer(r'\w+')
  wlist = tokenizer.tokenize(txt1)
  nt = len(wlist)
  nw.append(nt)
# text 2
  t2 = se['Recognized'][i]
  txt2 = t2.lower()
  tokenizer = RegexpTokenizer(r'\w+')
  wlist1 = tokenizer.tokenize(txt2)
  nt1 = len(wlist1)
  rw.append(nt1)

# Recognition rate

  from tables.idxutils import calc_chunksize
  from nltk.downloader import ErrorMessage
# txt1(original text), txt2 (recognized text)
 
  mword = []
  score = 0
  for i in range(0, len(wlist1)):
      w = wlist1[i]

      if w in wlist:
        sc = 1
      else:
        sc = 0
        mword.append(w)
      mwords = ' '.join(mword)
      score = score + sc
  mw.append(mwords)
  missingword = (score/len(wlist))*100

  # RecRate = float(format(missingword, '.2f'))
  # ErrRate = float(format((100.0 - RecRate), '.2f'))
  
  rr.append(missingword)
  # print('Matching words: %d'%score, 'out of %d words'%len(wlist))
  # print("="*50)
  # print('Recognition Rate: %f %%'%RecRate)
  # print('Error Rate: %f %%'%ErrRate)

se['NWord'] = nw
se['RecWord'] = rw
se['RecRate'] = rr
se['Missingwords'] = mw

se.to_csv('SETR_result.csv')

se.tail()


,SN,Filename,Runtime,Sentence,Recognized,NWord,RecWord,RecRate,Missingwords
14,S15,SE15.wav,0.261,Many complicated ideas about the rainbow have ...,Many complicated ideas about the rainbow have...,9,9,100.000000,
15,S16,SE16.wav,0.429,The difference in the rainbow depends consider...,The difference in the rainbow depends conside...,28,28,96.428571,coloured
16,S17,SE17.wav,0.345,The actual primary rainbow observed is said to...,The actual primary rainbow observed is said t...,19,18,89.473684,superimposition
17,S18,SE18.wav,0.464,If the red of the second bow falls upon the gr...,If the red of the second bow falls upon the g...,36,36,100.000000,
18,S19,SE19.wav,0.365,"This is a very common type of bow, one showing...","This is a very common type of bow, one showin...",21,21,100.000000,


The end